In [1]:
from geopy import distance
import numpy as np
import pandas as pd
import random
import os

csv_file = r"C:\Users\melvin.derouk\VINCI Energies\VF DAUPHINE SAVOIE - METHODES\16-Melvin DEROUCK\P8 - Accompagner le passage à l'électrique du parc auto\INVOXIA\export Invoxia\export_2023-07-03_2023-09-30.csv"

df_nom= pd.read_csv(r"C:\Users\melvin.derouk\VINCI Energies\VF DAUPHINE SAVOIE - METHODES\16-Melvin DEROUCK\P8 - Accompagner le passage à l'électrique du parc auto\CSV\Affectation trackers.csv", sep=";")

df = pd.read_csv(csv_file)

df_sliced_dict = {}


df=df.sort_values(by=['name','datetime'])

df_merged = pd.merge(df, df_nom[['name', 'nom']], left_on='name', right_on='name', how='left')

df_merged['name']

valeurs_a_conserver = ["b'T 1'", "b'T 2'", "b'T 3'", "b'T 4'", "b'T 5'", "b'T 6'", "b'T11'", "b'T12'"]

masque = df_merged['name'].isin(valeurs_a_conserver)

df_filtre = df_merged[masque]

for serial in df['name'].unique():
    
    df_sliced_dict[serial] = df[df['name'] == serial ]
  
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')
    
    df['lon-start'] = df['lng']
    df['lon-start'].loc[-1] = np.nan
    df['lon-start'] = np.roll(df['lon-start'], 1)
    df['lat-start'] = df['lat']
    df['lat-start'].loc[-1] = np.nan
    df['lat-start'] = np.roll(df['lat-start'], 1)
    df['time-start'] = df['datetime']
    df['time-start'].loc[-1] = np.nan
    df['time-start'] = np.roll(df['time-start'], 1)
    df = df.fillna(method='bfill')

    
    df['datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%dT%H:%M:%S")
    df['time-start'] = pd.to_datetime(df['time-start'])

    
    df['distance_dis_2d'] = df.apply(lambda x: distance.distance((x['lat-start'], x['lon-start']), (x['lat'], x['lng'])).km, axis = 1)
    df['time_delta_minutes'] = df.apply(lambda x: (x['datetime'] - x['time-start']).total_seconds()/60, axis=1) 

In [2]:
df['datetime']=(df['datetime'].dt.to_period('D'))

df= df[(df['time_delta_minutes']>0)]  
df=df[(df['stationary']=='move')]

df.insert(2, 'nom', df['name'].map(df_nom.set_index('name')['nom']))

C:\Users\melvin.derouk\AppData\Local\Temp\ipykernel_18664\1052932509.py:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['datetime']=(df['datetime'].dt.to_period('D'))


In [3]:
jour_astreinte = {
    "2023-07-08", "2023-07-09", "2023-07-14",
    "2023-07-15", "2023-07-16", "2023-07-22",
    "2023-07-23", "2023-07-29", "2023-07-30",
    "2023-08-05", "2023-08-06", "2023-08-12",
    "2023-08-13", "2023-08-15", "2023-08-19", 
    "2023-08-20", "2023-08-26", "2023-08-27", 
    "2023-09-02", "2023-09-03", "2023-09-09",
    "2023-09-10", "2023-09-16", "2023-09-17", 
    "2023-09-23", "2023-09-24", "2023-09-30"
    }

In [4]:
df['jours_astreinte'] = df['datetime'].dt.strftime("%Y-%m-%d").isin(jour_astreinte)

In [5]:
df_temp = df.drop_duplicates(subset=['nom', 'datetime'])

resultats = df_temp.groupby('nom')['jours_astreinte'].sum().reset_index()

resultats

,nom,jours_astreinte
0,ALIMI,1
1,BOURBOTTE,1
2,COLLI,0
3,LAROCHE,0
4,MARINELLO,1
5,MATHELIN,2
6,MENESES G.,1
7,OCCULI,5


In [6]:
astreinte_oculi = df[(df['nom']=="OCCULI") & (df['jours_astreinte'])]
astreinte_alimi = df[(df['nom']=="ALIMI") & (df['jours_astreinte'])]
astreinte_bourbotte = df[(df['nom']=="BOURBOTTE") & (df['jours_astreinte'])]
astreinte_marinello = df[(df['nom']=="MARINELLO") & (df['jours_astreinte'])]
astreinte_mathelin = df[(df['nom']=="MATHELIN") & (df['jours_astreinte'])]
astreinte_meneses = df[(df['nom']=="MENESES G.") & (df['jours_astreinte'])]

ind_excl = astreinte_oculi.index.tolist() + astreinte_alimi.index.tolist() + astreinte_bourbotte.index.tolist() + astreinte_marinello.index.tolist() + astreinte_mathelin.index.tolist() + astreinte_meneses.index.tolist()

In [8]:
df = df.drop(ind_excl)

In [9]:
verif_astreinte = df_temp[(df_temp['nom'] == 'MATHELIN') & (df_temp['jours_astreinte'])]

verif_astreinte

,serial,name,nom,datetime,lat,lng,accuracy,method,stationary,lon-start,lat-start,time-start,distance_dis_2d,time_delta_minutes,jours_astreinte
9572,18b79efff00d7b3a,b'T 6',MATHELIN,2023-07-08,45.56248,5.86510,32,PositionMethodGPS,move,5.86532,45.56246,2023-07-08 14:55:14.595000+00:00,0.017319,61.278333,True
9051,18b79efff00d7b3a,b'T 6',MATHELIN,2023-07-22,45.56235,5.86473,25,PositionMethodNVXBSSID,move,5.87020,45.56454,2023-07-21 19:59:53.303000+00:00,0.491540,772.552767,True


In [10]:
coeff_list = {"ALIMI": 1.3, "BOURBOTTE": 1.24, "MENESES G.": 1.3, "LAROCHE": 1.3, "OCCULI": 1.3, "MATHELIN": 1.3, "MARINELLO": 1.35, "COLLI": 1.3}

In [11]:
dfinale = df.groupby(['datetime', 'nom'])['distance_dis_2d'].agg(['count'])

dfinale['kilometre parcourus']= df.groupby(['datetime','nom'])['distance_dis_2d'].agg(['sum']).round(0)
dfinale['temps en minutes']= df.groupby(['datetime','nom'])['time_delta_minutes'].agg(['sum']).round(0)

In [12]:
dfinale = dfinale.reset_index()

In [13]:
dfinale['coeff'] = dfinale['nom'].map(coeff_list)

dfinale['kilometres corrigés'] = dfinale['kilometre parcourus']*dfinale['coeff']

In [14]:
dfinale['-100km']=np.where(dfinale['kilometres corrigés'] < 100, 1, 0)
dfinale['100-150km'] = np.where((dfinale['kilometres corrigés'] >= 100) & (dfinale['kilometres corrigés'] < 150), 1, 0)
dfinale['150-200km'] = np.where((dfinale['kilometres corrigés'] >= 150) & (dfinale['kilometres corrigés'] < 200), 1, 0)
dfinale['200-250km']= np.where((dfinale['kilometres corrigés'] >= 200) & (dfinale['kilometres corrigés'] < 250), 1, 0)
dfinale['250-300km']= np.where((dfinale['kilometres corrigés'] >= 250) & (dfinale['kilometres corrigés'] < 300), 1, 0)
dfinale['+ de 300km']=np.where(dfinale['kilometres corrigés']>=300,1,0)


dfinale=dfinale[(dfinale['kilometres corrigés']>3)]

In [26]:
df[df['jours_astreinte']==True]

,serial,name,nom,datetime,lat,lng,accuracy,method,stationary,lon-start,lat-start,time-start,distance_dis_2d,time_delta_minutes,jours_astreinte


# Table jours réels

In [31]:
total_jours = df.groupby('nom')['datetime'].nunique().reset_index()

total_jours.columns = ['nom', 'total_jours']

total_jours

,nom,total_jours
0,ALIMI,39
1,BOURBOTTE,61
2,COLLI,46
3,LAROCHE,41
4,MARINELLO,46
5,MATHELIN,35
6,MENESES G.,62
7,OCCULI,44


In [32]:
df_jours = pd.merge(total_jours, resultats, on='nom')

df_jours

,nom,total_jours,jours_astreinte
0,ALIMI,39,1
1,BOURBOTTE,61,1
2,COLLI,46,0
3,LAROCHE,41,0
4,MARINELLO,46,1
5,MATHELIN,35,2
6,MENESES G.,62,1
7,OCCULI,44,5


In [33]:
df_jours["jours_réels"] = df_jours['total_jours'] - df_jours['jours_astreinte']

df_jours

,nom,total_jours,jours_astreinte,jours_réels
0,ALIMI,39,1,38
1,BOURBOTTE,61,1,60
2,COLLI,46,0,46
3,LAROCHE,41,0,41
4,MARINELLO,46,1,45
5,MATHELIN,35,2,33
6,MENESES G.,62,1,61
7,OCCULI,44,5,39


In [34]:
df_jours['% var'] = df_jours['jours_astreinte']/df_jours['total_jours']*100

df_jours['% var'] = df_jours['% var'].round(1)

df_jours.sort_values(by='% var', ascending=False, inplace=True)

df_jours

,nom,total_jours,jours_astreinte,jours_réels,% var
7,OCCULI,44,5,39,11.4
5,MATHELIN,35,2,33,5.7
0,ALIMI,39,1,38,2.6
4,MARINELLO,46,1,45,2.2
1,BOURBOTTE,61,1,60,1.6
6,MENESES G.,62,1,61,1.6
2,COLLI,46,0,46,0.0
3,LAROCHE,41,0,41,0.0


In [35]:
jours_decharges_mapping = {"LAROCHE": 8, "MATHELIN": 11, "ALIMI": 10, "BOURBOTTE": 0, "COLLI": 0, "MARINELLO": 0, "MENESES G.": 0, "OCCULI": 0}

df_jours['jours_déchargés'] = df_jours['nom'].map(jours_decharges_mapping)

df_jours

,nom,total_jours,jours_astreinte,jours_réels,% var,jours_déchargés
7,OCCULI,44,5,39,11.4,0
5,MATHELIN,35,2,33,5.7,11
0,ALIMI,39,1,38,2.6,10
4,MARINELLO,46,1,45,2.2,0
1,BOURBOTTE,61,1,60,1.6,0
6,MENESES G.,62,1,61,1.6,0
2,COLLI,46,0,46,0.0,0
3,LAROCHE,41,0,41,0.0,8


In [36]:
jours_congés_mapping = {"LAROCHE": 7, "MATHELIN": 10, "ALIMI": 10, "BOURBOTTE": 2, "COLLI": 15, "MARINELLO": 15, "MENESES G.": 0, "OCCULI": 14}

df_jours['jours_congés'] = df_jours['nom'].map(jours_congés_mapping)

df_jours

,nom,total_jours,jours_astreinte,jours_réels,% var,jours_déchargés,jours_congés
7,OCCULI,44,5,39,11.4,0,14
5,MATHELIN,35,2,33,5.7,11,10
0,ALIMI,39,1,38,2.6,10,10
4,MARINELLO,46,1,45,2.2,0,15
1,BOURBOTTE,61,1,60,1.6,0,2
6,MENESES G.,62,1,61,1.6,0,0
2,COLLI,46,0,46,0.0,0,15
3,LAROCHE,41,0,41,0.0,8,7
